In [1]:
#standard package
import pandas as pd
import numpy as np
import os
import math
from collections import Counter
import time 
import glob
import tqdm
import sys
import shutil
import itertools
import pickle
import operator
from operator import itemgetter
import datetime as dt

#topics modeling
import pyLDAvis
import pyLDAvis.gensim
import gensim 
from gensim import corpora, models, similarities

#plot
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

c:\users\camil\anaconda3\lib\site-packages\_pytest\fixtures.py:844: DeprecationWarning: The `convert` argument is deprecated in favor of `converter`.  It will be removed after 2019/01.
  params = attr.ib(convert=attr.converters.optional(tuple))
c:\users\camil\anaconda3\lib\site-packages\_pytest\fixtures.py:846: DeprecationWarning: The `convert` argument is deprecated in favor of `converter`.  It will be removed after 2019/01.
  ids = attr.ib(default=None, convert=_ensure_immutable_ids)
c:\users\camil\anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
from UTILS import word_from_MLP, word_from_MLP_end_and_begin_atlongDuration
import config_mobility as config

c:\users\camil\anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


we restrict the time series to one value per 60 seconds 
we compute the complexity variables each 30 minutes 
each variables includes the values of at least the last 120.00 minutes (i.e. are using 120.00 values)


In [3]:
#comment on this run of the script
comment = ''

### choose hybrid type
type_hybrid = 'LB' #LSL, all, LB

### choose type of word definition
type_ = 'fixedLength' ; length_words = 3 ; comment = comment+'lengthwords_'+str(length_words)
#type_ = 'varyinglength' ; morethan4_in1 = True ; comment = comment+'_morethan4in1_'+str(int(morethan4_in1))

### choose session to take into account
#Sess2keep = [1,2,3] 
Sess2keep = [2,3,4] 
Sess2keep = [3,4,5] 
Sess2keep = [4,5,6] 
Sess2keep = [5,6,7] 
Sess2keep = [6,7,8] 
Sess2keep = [7,8,9] 
Sess2keep = [8,9,10] 
Sess2keep = [9,10,11]

### unique naming
title_ = type_hybrid+'_'+type_+'_'+comment+'_sess_'+'_'.join([str(x) for x in Sess2keep]) 
#'_LB', '_all','_LSL' '_LB_newword_def' '_all_newword_def'
print('This topic modeling will be save under the name: '+title_)

This topic modeling will be save under the name: LB_fixedLength_lengthwords_3_sess_9_10_11


In [4]:
#initialise parameters
path_extracted_data = config.path_extracted_data
path_initial_data = config.path_initial_data
id_run = config.id_run
#create a director if not existing
path_save = os.path.join(path_extracted_data,'visual','LDA', title_)
if not os.path.exists(path_save):
    os.makedirs(path_save)

# Download data

In [5]:
df_master = pd.read_csv(os.path.join(path_initial_data,'Mo_Masterfile.csv'), sep=';',parse_dates=['day']) #, parse_dates=['date']) wrong!
df_master.rename(columns={'time':'Timestamp','day':'DayID','hen':'HenID','date':'day'}, inplace=True)
df_master['HenID'] = df_master['HenID'].map(lambda x: 'hen_'+str(x))
print(df_master.shape)
display(df_master.head(3))
#dico_henid_hybrid = dict(zip(df_master['HenID'].tolist(),df_master['hybrid'].tolist()))
li_LB = df_master[df_master['hybrid']=='LB']['HenID'].unique()
li_LSL = df_master[df_master['hybrid']=='LSL']['HenID'].unique()
print(len(li_LB), len(li_LSL))
li_LB[0:2]

(7920, 43)


,HenID,timepoint,group,age,DayID,day,file,severity,pen,hybrid,...,meanchange,mean1,medianchange,transitions,real0,sumdur,nest_vs_total,severity_diff,KBF_new,indicator
0,hen_1,1,A,21,1,27.10.2016,1A01,0.0,1,LSL,...,1.23,1.0,1.0,91.0,NaN,57067.0,0.177419,NaN,0.0,NaN
1,hen_1,1,A,21,2,28.10.2016,1A01,0.0,1,LSL,...,1.28,1.0,1.0,108.0,NaN,56854.0,0.171053,0.0,0.0,NaN
2,hen_1,1,A,21,3,29.10.2016,1A01,0.0,1,LSL,...,1.17,1.0,1.0,128.0,NaN,56721.0,0.255319,0.0,0.0,NaN


60 60


array(['hen_41', 'hen_42'], dtype=object)

In [6]:
#open time series per session and compute the variables for each session (car time series make sence at session level), et en 
#plus des variables tel que running entropy over the whole session ateach last timestamp of each level make sence only at 
#session level
START_TIME = time.clock()
starting_hour = 2
li_path = glob.glob(os.path.join(path_extracted_data, id_run+'_TimeSeries_*.csv'))
li_path = [i for i in li_path if '_hen_' not in i]
li_df = []
for path_ in tqdm.tqdm(li_path):
    
    #download time series associated to this session
    name_ = path_.split('_')[-1].split('.')[0]
    print('-------------------------------------------------------------------------', name_)
    df_ts = pd.read_csv(path_, sep=';', parse_dates=['Timestamp', 'day']) 
    li_hours_to_consider=list(range(starting_hour,17))
    df_ts['is_day'] = df_ts['hour'].map(lambda x: x in li_hours_to_consider)
    print(df_ts.shape)
    df_ts = df_ts[df_ts['is_day']]
    print(df_ts.shape)
    df_ts['session'] = name_
    li_df.append(df_ts)
df_ts = pd.concat(li_df)
print(df_ts.shape)
display(df_ts.head(2))
display(df_ts.tail(2))    
END_TIME = time.clock()
print ("Total running time: %.2f mn" %((END_TIME-START_TIME)/60))  

  0%|                                                                                           | 0/22 [00:00<?, ?it/s]

------------------------------------------------------------------------- 10A
(513000, 54)
(324000, 54)


  5%|███▊                                                                               | 1/22 [00:05<02:03,  5.86s/it]

------------------------------------------------------------------------- 10B


c:\users\camil\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,45,46,47,48,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


...

100%|██████████████████████████████████████████████████████████████████████████████████| 22/22 [02:13<00:00,  6.26s/it]


(7128000, 126)


,Timestamp,day,hen_1,hen_10,hen_100,hen_101,hen_102,hen_103,hen_104,hen_105,...,hen_94,hen_95,hen_96,hen_97,hen_98,hen_99,hour,is_day,nbr_nan,session
1800,2017-07-06 02:00:00,2017-07-06,zone_5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2,True,0,10A
1801,2017-07-06 02:00:01,2017-07-06,zone_5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2,True,0,10A


,Timestamp,day,hen_1,hen_10,hen_100,hen_101,hen_102,hen_103,hen_104,hen_105,...,hen_94,hen_95,hen_96,hen_97,hen_98,hen_99,hour,is_day,nbr_nan,session
487798,2017-06-14 16:59:58,2017-06-14,NaN,NaN,NaN,zone_5,NaN,zone_2,zone_2,zone_3,...,zone_3,zone_4,zone_5,zone_5,zone_4,zone_2,16,True,0,9B
487799,2017-06-14 16:59:59,2017-06-14,NaN,NaN,NaN,zone_5,NaN,zone_2,zone_2,zone_3,...,zone_3,zone_4,zone_5,zone_5,zone_4,zone_2,16,True,0,9B


Total running time: 2.94 mn


In [7]:
#adapt to the need (keep columns of one species only if wanted and rows with session that we want)
print(df_ts.shape)
if 'LB' in title_:
    df_ts = df_ts.filter([x for x in df_ts.columns if (not x.startswith('hen_')) | (x in li_LB)],axis=1)
if 'LSL' in title_:
    df_ts = df_ts.filter([x for x in df_ts.columns if (not x.startswith('hen_')) | (x in li_LSL)],axis=1)
print(df_ts.shape)
df_ts['sessionID'] = df_ts['session'].map(lambda x: int(x[:-1]))
#df_ts['sessionID'].value_counts()
print(df_ts.shape)
df_ts = df_ts[df_ts['sessionID'].isin(Sess2keep)]

(7128000, 126)
(7128000, 66)
(7128000, 67)


# Topics modelling (lda - latent dirichlet allocation)

### define parameters

In [8]:
#define the fixed number of bouts included in each word
#define 3 size of duration length (small, medium, long) that will be associated to each movement additionally to the daily hour 
#dico_size = {'transition':range(1,60),
#            'zone':range(60,60*60*24)}
dico_size = {'small':range(1,60),
             'intermediate':range(60,15*60),
             'large':range(15*60,60*60*24)}
li_hen = [i for i in df_ts.columns if i.startswith('hen_')]

In [9]:
all_ = False
        
### documents:  set of words of daily hen time series
documents = {}
for day, df_ in tqdm.tqdm(df_ts.groupby(['day'])):
    df_ = df_.fillna(' ')
    for h in li_hen:
        li = df_[h].tolist()
        #remove if nan in ts or if only one zone (we separate for efficiency)
        if li[0]!=' ':
            if len(set(li))>1:
                if type_=='varyinglength':
                    documents[h+'/-/'+str(day).split(' ')[0]] = word_from_MLP_end_and_begin_atlongDuration(li, 60*5, 
                                                                                                           starting_hour,
                                                                                                           dico_size, config,
                                                                                                           morethan4_in1=morethan4_in1)
                if type_ =='fixedLength':
                    documents[h+'/-/'+str(day).split(' ')[0]] = word_from_MLP(li, length_words, starting_hour, dico_size)
                    

100%|██████████████████████████████████████████████████████████████████████████████████| 36/36 [00:19<00:00,  2.39it/s]


In [10]:
h = 'hen_45'
day = '2016-10-28'
display(documents[h+'/-/'+day])
plt.plot(df_ts[df_ts['day']==dt.datetime.strptime(day, '%Y-%m-%d')][h].tolist());
plt.title(h+'   '+day);

KeyError: 'hen_45/-/2016-10-28'

In [ ]:
li_words = []
for h_day, li_li_tupleWordTime in documents.items():
    li_words.extend([x for x in li_li_tupleWordTime])
print(len(li_words), len(set(li_words)))

#compute frequencies
c_words = Counter(li_words)

In [ ]:
len([x for x,c in c_words.items() if c==1]), [x for x,c in c_words.items() if c==1]

In [ ]:
### frequence analysis
#create a list of all words
li_words = []
for h_day, li_li_tupleWordTime in documents.items():
    li_words.extend([x for x in li_li_tupleWordTime])
print(len(li_words), len(set(li_words)))

#compute frequencies
c_words = Counter(li_words)
df_word_frequence = pd.DataFrame.from_dict({'word':list(c_words.keys()),'frequence':list(c_words.values())})
df_word_frequence = df_word_frequence.sort_values('frequence',ascending=False)
df_word_frequence.to_csv(os.path.join(path_save,'word_Frequence.csv'),index=False,sep=';')
display(df_word_frequence.head(3))
display(df_word_frequence.tail(3))

#simple barplot (sorted with x values)
d = {k:v for k,v in c_words.items() if v>5}
d = sorted(d.items(), key=operator.itemgetter(1))
x = [i[0] for i in d]
y = [i[1] for i in d]
fig = plt.figure(figsize=(30,7))
ax = plt.subplot(111)
width = 0.8
ax.bar(range(len(x)), y, width=width)
plt.title('most frequent words')
ax.set_xticks(np.arange(len(x)) + width/2)
ax.set_xticklabels(x, rotation=90,size=6);
plt.savefig(os.path.join(path_save,'most_frequent_word.png'),dpi=300,format='png',bbox_inches='tight')
plt.show()

#histogram
plt.hist(df_word_frequence[df_word_frequence['frequence']<=100]['frequence'],bins='auto') #into 15 equal parts 
#return: [0]: vector of length bins with #elements in each bins
#and [1]: when the vectors starts (for plot)
plt.xlabel('frequencies')
plt.ylabel('number of words')    
#--> choose the nbr_times parameter: number of times a words need to appear at least this amount of time, in the overall set of 
#documents to be taken into account
plt.savefig(os.path.join(path_save,'histogram_of_word_frequencies.png'),dpi=300,format='png',bbox_inches='tight')
plt.show()
nbr_times = 2

In [ ]:
### clean documents
def MLPsWord4lda(documents, c_words, nbr_times):
    #remove all words appearing less or equal to nbr_times times in the overall time series
    tokens_removed = set(word for word in c_words.keys() if c_words[word]<nbr_times)
    cleaned_documents = {h_day:[word for word in words if word not in tokens_removed] for h_day,words in documents.items()}
    #perhaps later: remove with to much transition words? keep it for now, I dont know what this means
    return(cleaned_documents, tokens_removed)

cleaned_documents, tokens_removed = MLPsWord4lda(documents, c_words, nbr_times)
len(tokens_removed)

### dictionary & bag of word corpus
#dictionary:  mapping from word IDs to words
#keep track of the order of the li_documents regarding the ts ID
dico_tsID_listID = dict(zip(list(cleaned_documents.keys()),range(len(cleaned_documents.keys()))))
dico_listID_tsID = {v:k for k,v in dico_tsID_listID.items()}
li_documents = [cleaned_documents[dico_listID_tsID[listID]] for listID in range(len(dico_listID_tsID))]
print('We have %d documents (i.e. daily hens time series). The first one has %d words (once cleaned)'%(len(li_documents), 
                                                                                                       len(li_documents[0])))
dictionary = corpora.Dictionary(li_documents)
print('There is %d words in your dictionary'%len(dictionary))

#corpus: list with a bag of words (sparse document vectors: list of tuples(word_id, appearance)) for each documents
corpus = [dictionary.doc2bow(text) for text in li_documents]
print(len(corpus),corpus[10])

#save
pickle.dump(corpus, open(os.path.join(path_save,'corpus.pkl'), 'wb'))
dictionary.save(os.path.join(path_save,'dictionary.gensim'))

In [ ]:
for nbr_topics in tqdm.tqdm(range(2,15)): 

    #create a director if not existing
    path_save = os.path.join(path_extracted_data,'visual','LDA', title_, str(nbr_topics))
    if not os.path.exists(path_save):
        os.makedirs(path_save)
        
    #train lda
    start = time.time()
    lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, 
                                   num_topics=nbr_topics, 
                                   passes=30, 
                                   chunksize=50, 
                                   random_state=100,
                                   update_every=5, 
                                   alpha='auto', 
                                   per_word_topics=False)
    lda.save(os.path.join(path_save,'model'+str(nbr_topics)+'.gensim'))
    end = time.time()
    print ("Total running time: %dmin"%((end-start)/60))

    #show the latent topics
    for topicsID_topicsWordDistribution in lda.print_topics():
        print('\n----',topicsID_topicsWordDistribution[0])
        print(topicsID_topicsWordDistribution[1])
        
    #summarize the results
    df_topics = pd.DataFrame(list(documents.items()),columns=['documentID','li_words'])
    df_topics['li_words_cleaned'] = df_topics['documentID'].map(lambda x: cleaned_documents[x])
    df_topics['corpus'] = df_topics['li_words_cleaned'].map(lambda x: dictionary.doc2bow(x))
    df_topics['lda_corpus'] = df_topics['corpus'].map(lambda x: lda[x])

    #add info on hen
    df_topics['HenID'] = df_topics['documentID'].map(lambda x: x.split('/-/')[0])
    df_topics['day'] = df_topics['documentID'].map(lambda x: dt.datetime.strptime(x.split('/-/')[1], '%Y-%m-%d'))

    #add info of topic 
    df_topics['topic_info'] = df_topics['lda_corpus'].map(lambda x: sorted(x,key=itemgetter(1))[-1])
    df_topics['topic'] = df_topics['topic_info'].map(lambda x: x[0])
    df_topics['topic_proba'] = df_topics['topic_info'].map(lambda x: x[1])
    for t in range(nbr_topics):
        df_topics['topic_'+str(t)+'_proba'] = df_topics['lda_corpus'].map(lambda x: max([i[1] for i in x if i[0]==t]+[0]))

    #save
    df_topics.to_csv(os.path.join(path_save,'df_topics'+str(nbr_topics)+'.csv'), index=False,sep=';')
    print(df_topics.shape)
    df_topics.head(3)
    print(df_topics['topic'].value_counts())
    if len(df_topics['topic'].unique())!=nbr_topics:
        print('WARNING: to many topics, there is only %d instead of the %d wanted, we will stop now'%(len(df_topics['topic'].unique()), nbr_topics))
        sys.exit()
    ### topics repartition across hens
    df_plot = df_topics.groupby(['HenID','topic']).size().reset_index().pivot(columns='topic', index='HenID', values=0)
    df_plot = df_plot.fillna(0)
    display(df_plot.head(3))
    df_plot_normalized = df_plot.div(df_plot.sum(axis=1)*0.01, axis=0).sort_values([0]) #sort accoridng to subject 0
    display(df_plot_normalized.head(3))
    li_color = sns.color_palette("RdBu", nbr_topics)
    df_plot_normalized.plot(x=df_plot_normalized.index, kind='bar', stacked=True, figsize=(30,7), 
                            legend=True, color=li_color).legend(bbox_to_anchor=(1.2, 0.5));
    plt.savefig(os.path.join(path_save,'topic_repartition_across_hen_'+str(nbr_topics)+'.png'), 
                dpi=300, format='png', bbox_inches='tight')
    plt.show()
    
    ### topic mixture across documents    
    if all_:
        for t in range(nbr_topics):
            df_plot_normalized = df_topics[['topic_'+str(i)+'_proba' for i in range(nbr_topics)]].sort_values(['topic_'+str(t)+'_proba'])
            df_plot_normalized.plot(x=df_plot_normalized.index, kind='bar', stacked=True, figsize=(40,10), 
                                    legend=True).legend(bbox_to_anchor=(1.2, 0.5));
            plt.savefig(os.path.join(path_save, 'topic_mixture_across_document_'+str(t)+'.png'),dpi=300,format='png',bbox_inches='tight')
            plt.show()
        
    ### topics appearance across days
    df_plot = df_topics.sort_values('day', ascending=True).copy()
    #dico_topicID_color = {0:(0.979891, 0.90894778, 0.84827858), 1:'blue', 2:'yellow', 3:'green', 4:'grey', 5:'lime', 6:'orange'}
    li_color = sns.color_palette("rocket_r", nbr_topics)
    c = 2
    if nbr_topics%3==0:
        c = 3
    l = math.ceil(nbr_topics/c)
    fig = plt.figure(figsize=(c*5, l*5))
    x = df_plot['day'].unique()
    for i, (t, df_) in enumerate(df_plot.groupby(['topic'])):
        ax = plt.subplot(l,c,i+1)
        df_ = df_.groupby(['day'])['documentID'].count().reset_index()
        x = df_['day'].tolist()
        y = df_['documentID'].tolist()
        print(len(x))
        plt.plot(x,y,color=li_color[i])
        plt.ylim(0,max(y)+max(int(max(y)*0.1),1))
        for label in ax.get_xticklabels():
            label.set_rotation(25) 
        plt.title('topic   '+str(t))
    plt.savefig(os.path.join(path_save,'topic_appearance_across_days_.png'),dpi=300,format='png',bbox_inches='tight')
    plt.show()
    display(df_.head(3))    
    
    ### hens main topics across days
    if all_:
        df_plot = df_topics.sort_values('day', ascending=True).copy()
        #x = list(set(df_plot['day'].tolist()))
        for t in range(nbr_topics):
            fig = plt.figure(figsize=(20,6))
            for i, (h, df_) in enumerate(df_plot.groupby(['HenID'])):
                x = df_['day'].tolist()
                y = df_['topic_'+str(t)+'_proba'].tolist()
                plt.plot(x,y)
                plt.show()
                fig = plt.figure(figsize=(20,6))
                if i==5:
                    sys.exit()
            plt.title('topic   '+str(t))
            plt.show()
        
    ### main topics proba
    plt.hist(df_topics['topic_proba'].dropna(), bins=200)
    plt.xlabel('main topic predominance')
    plt.ylabel('number of documents (daily hen ts)');
    plt.savefig(os.path.join(path_save,'main_topic_proba.png'),dpi=300,format='png',bbox_inches='tight')
    plt.show()
    
    ### plot visual time series into cluster-folders
    for i in range(0,nbr_topics):
        path_ts = os.path.join(path_save, 'timeseries_plot_cluster', str(i))
        if not os.path.exists(path_ts):
            os.makedirs(path_ts)

    path_ = r'D:\vm_exchange\AVIFORUM\data\extracted_info_mobility_VF\visual\TimeSeriesPlot\time_series_plot'

    for i in tqdm.tqdm(range(df_topics.shape[0])):
        HenID = df_topics.iloc[i]['HenID']
        day = df_topics.iloc[i]['day']
        topic = df_topics.iloc[i]['topic']
        image_name = 'VF_'+str(day).split(' ')[0]+'_'+HenID+'.png'
        shutil.copy(os.path.join(path_, image_name), 
                    os.path.join(path_save, 'timeseries_plot_cluster', str(topic), image_name))
        
    #LDA nice visual
    lda_model = gensim.models.ldamodel.LdaModel.load(os.path.join(path_save, 'model'+str(nbr_topics)+'.gensim'))
    lda_display = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary, sort_topics=False)
    pyLDAvis.display(lda_display)
    pyLDAvis.save_html(lda_display, os.path.join(path_save, str(nbr_topics)+'_lda.html'))

In [ ]:
#df_summary = df_topics.groupby(['topic'])['HenID','day'].agg(lambda x: Counter(x)).reset_index()

# TODO